In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("DataCleaning") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [4]:
adult_train_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("../sources/adult.data") \

In [5]:
adult_test_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("../sources/adult.test") \

## Veri Birleştirme ##

In [6]:
adult_whole_df = adult_train_df.union(adult_test_df)
adult_whole_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


## 1. Do trim for every column 

In [7]:
from pyspark.sql.functions import *

In [9]:
adult_whole_df1 = adult_whole_df \
.withColumn("workclass", trim(col("workclass"))) \
.withColumn("education", trim(col("education"))) \
.withColumn("marital_status", trim(col("marital_status"))) \
.withColumn("occupation", trim(col("occupation"))) \
.withColumn("relationship", trim(col("relationship"))) \
.withColumn("race", trim(col("race"))) \
.withColumn("sex", trim(col("sex"))) \
.withColumn("native_country", trim(col("native_country"))) \
.withColumn("output", trim(col("output"))) \

In [10]:
adult_whole_df.count()

48842

In [11]:
adult_whole_df1.count()

48842

 ## 2. OUTPUT  "." CLEAR
    

In [13]:
adult_whole_df2 = adult_whole_df1 \
.withColumn("output", regexp_replace(col("output"), "<=50K.","<=50K")) \
.withColumn("output", regexp_replace(col("output"), ">50K.",">50K"))

In [14]:
adult_whole_df2.groupBy(col("output")).agg({"*":"count"}) \
.toPandas().head()

,output,count(1)
0,<=50K,37155
1,>50K,11687


## 3. NULL CONTROL

In [15]:
adult_whole_df2.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'output']

In [18]:
sayac_for_null = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).isNull()).count() > 0 ):
        print(sayac_for_null, ". ", sutun, " içinde null var.")
    else:
        print(sayac_for_null, sutun)
    sayac_for_null += 1

1 age
2 workclass
3 fnlwgt
4 education
5 education_num
6 marital_status
7 occupation
8 relationship
9 race
10 sex
11 capital_gain
12 capital_loss
13 hours_per_week
14 native_country
15 output


## 4. "?" CONTROL

In [20]:
sayac_for_question = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).contains("?")).count() > 0 ):
        print(sayac_for_question, ". ", sutun, " içinde ? var.")
    else:
        print(sayac_for_question, sutun)
    sayac_for_question += 1

1 age
2 .  workclass  içinde ? var.
3 fnlwgt
4 education
5 education_num
6 marital_status
7 .  occupation  içinde ? var.
8 relationship
9 race
10 sex
11 capital_gain
12 capital_loss
13 hours_per_week
14 .  native_country  içinde ? var.
15 output


In [21]:
adult_whole_df2.select("workclass","occupation","native_country","output") \
.filter(
    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?")
    
) \
.groupBy("workclass","occupation","native_country","output").count() \
.orderBy(col("count").desc()) \
.toPandas().head(50)

,workclass,occupation,native_country,output,count
0,?,?,United-States,<=50K,2284
1,?,?,United-States,>50K,246
2,Private,Other-service,?,<=50K,100
3,Private,Sales,?,<=50K,55
4,Private,Prof-specialty,?,<=50K,51
5,Private,Craft-repair,?,<=50K,48
6,Private,Prof-specialty,?,>50K,48
7,?,?,Mexico,<=50K,48
8,Private,Adm-clerical,?,<=50K,47
9,Private,Machine-op-inspct,?,<=50K,42


In [22]:
adult_whole_df3 = adult_whole_df2 \
.filter(~(
    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?")
))

adult_whole_df3.count()

45222

In [29]:
## ANOTHER WAY WİTH NATIVE SQL 
adult_whole_df2.createOrReplaceTempView("df2")

df3 = spark.sql(" SELECT * from df2\
                where (workclass NOT LIKE '%?%') and (occupation NOT LIKE '%?%') and (native_country NOT LIKE '%?%')")
df3.count()

45222

## 5 -  WEAK CLASSES TO REMOVE ##

In [32]:
adult_whole_df4 = adult_whole_df3 \
.filter(~(
    col("workclass").contains("never-worked") | col("workclass").contains("without-pay") | 
       col("occupation").contains("Armed-Forces") | col("native_country").contains("Holand-Netherlands")
))

print(adult_whole_df3.count())
print(adult_whole_df4.count())

45222
45207
